In [4]:
#coding=utf-8
import numpy as np
import sys,os
import glob
from math import log10
from nltk.corpus import wordnet as wn
import string
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

stopwords=[line.strip() for line in open('stopwords','r').readlines()]

def labelmax(p):
    max_prob=0
    for label,prob in p.items():
        if max_prob<prob:
            prob=max_prob
            max_label=label
    return max_label

def clean(sent):
    delstr='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t'
    identify = string.maketrans(delstr,' '*len(delstr))
    return sent.translate(identify)
    # return sents

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass

    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

def stem(word):
    try:
        res=wn.morphy(word)
    except:
        res=''
    if res!=None and res not in stopwords and not is_number(res):
        return res.encode('ascii')
    else:
        return ''


def count(docs,word):
    total=0
    for doc in docs:
        total+=doc.count(word)
    return total
root='20_newsgroups'
train_set={}
test_set={}
text_labels=[]
text_count=[]
newdir='all'
for filedir in glob.glob(os.path.join(root, '*')):
    #text labels and sizes
    label = filedir.split('/')[-1]
    text_labels.append(label)
    files=glob.glob(os.path.join(filedir, '*'))
    size=len(files)
    text_count.append(size)

    #departure of train and test sets
    train_set[label]=[]
    test_set[label]=[]
    indices = np.random.permutation(size)
    for i in indices[:int(size*0.8)]:
        lines=open(files[i],'r').readlines()
        for i,line in enumerate(lines):
            if line.startswith('Lines'):
                ind=i
                break
        seq = [stem(word) for word in ' '.join([clean(line.strip().lower()) for line in lines[ind+1:]]).strip().split() if stem(word)!='']
        # print seq
        train_set[label].append(seq)
    for i in indices[int(size*0.8):]:
        lines=open(files[i],'r').readlines()
        for i,line in enumerate(lines):
            if line.startswith('Lines'):
                ind=i
                break
        seq = [stem(word) for word in ' '.join([clean(line.strip().lower()) for line in lines[ind+1:]]).strip().split() if stem(word)!='']
        # print seq
        test_set[label].append(seq)

word_bank=set()
for doc in train_set.values():
    for words in doc:
        word_bank|=set(words)
print len(word_bank)

25849


In [5]:
labels=train_set.keys()
print labels
corpus=[]
label_inds=[]
for label,docs in train_set.items():
    for doc in docs:
        label_inds.append(labels.index(label))
        corpus.append(' '.join(doc))
for label,docs in test_set.items():
    for doc in docs:
        label_inds.append(label)
        corpus.append(' '.join(doc))
print len(corpus)


['sci.crypt', 'comp.sys.mac.hardware', 'talk.politics.misc', 'soc.religion.christian', 'rec.motorcycles', 'sci.med', 'comp.graphics', 'comp.windows.x', 'comp.sys.ibm.pc.hardware', 'talk.politics.guns', 'alt.atheism', 'comp.os.ms-windows.misc', 'sci.space', 'talk.religion.misc', 'misc.forsale', 'rec.sport.hockey', 'rec.sport.baseball', 'talk.politics.mideast', 'rec.autos', 'sci.electronics']
19997


In [6]:
label_inds=np.array(label_inds)
vectorizer = CountVectorizer()
tfidf = TfidfTransformer()
X = vectorizer.fit_transform(corpus)
print len(vectorizer.get_feature_names())
transformer =  TfidfTransformer()  
tfidf = transformer.fit_transform(X)  
print tfidf.toarray().shape
print label_inds.shape

27067
(19997, 27067)
(19997,)


In [7]:
XX = tfidf.toarray()[:int(len(corpus)*0.4)]
label_inds = label_inds[:int(len(corpus)*0.4)]
print XX.shape
test_split_mask = np.random.rand(len(XX)) < 0.8
train_X = XX[test_split_mask]
train_Y = label_inds[test_split_mask]
test_X = XX[~test_split_mask]
test_Y = label_inds[~test_split_mask]
print train_X.shape, test_Y.shape

(7998, 27067)
(6420, 27067) (1578,)


In [8]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=1)
clf.fit(train_X,train_Y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')

In [9]:
clf.score(test_X,test_Y)

0.63371356147021551

In [ ]:
from sklearn.svm import SVC
print clf

In [ ]:
clf=SVC()
clf.fit(train_X,train_Y)